In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:

zip_ref = zipfile.ZipFile("/content/drive/My Drive/10kArticles.zip", 'r')
zip_ref.extractall("/tmp/")
zip_ref.close()

articleLoc = '/tmp/ArticleFetching'

In [ ]:
pip install transformers

     |████████████████████████████████| 4.2 MB 7.0 MB/s 
     |████████████████████████████████| 6.6 MB 43.1 MB/s 
     |████████████████████████████████| 596 kB 45.0 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import AutoTokenizer, AutoConfig, TFAutoModel
import os

In [ ]:
 
MODEL_NAME = 'allenai/longformer-base-4096'
MAX_LEN = 1024

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('model')

config = AutoConfig.from_pretrained(MODEL_NAME) 
config.save_pretrained('model')

backbone = TFAutoModel.from_pretrained(MODEL_NAME, config=config)
backbone.save_pretrained('model')
DOWNLOADED_MODEL_PATH = '/content/model'

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/729M [00:00<?, ?B/s]

Some layers from the model checkpoint at allenai/longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at allenai/longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.


In [ ]:
def build_model():
    
    tokens = tf.keras.layers.Input(shape=(MAX_LEN,), name = 'tokens', dtype=tf.int32)
    attention = tf.keras.layers.Input(shape=(MAX_LEN,), name = 'attention', dtype=tf.int32)
    
    config = AutoConfig.from_pretrained(DOWNLOADED_MODEL_PATH+'/config.json') 
    backbone = TFAutoModel.from_pretrained(DOWNLOADED_MODEL_PATH+'/tf_model.h5', config=config)
    
    x = backbone(tokens, attention_mask=attention)
    
    model = tf.keras.Model(inputs=[tokens,attention], outputs=x)
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 1e-4),
                  loss = [tf.keras.losses.CategoricalCrossentropy()],
                  metrics = [tf.keras.metrics.CategoricalAccuracy()])
    
    return model

In [ ]:
model = build_model()

All model checkpoint layers were used when initializing TFLongformerModel.

All the layers of TFLongformerModel were initialized from the model checkpoint at /content/model/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:

feature_array = []
location = os.listdir(articleLoc)
count = 0

articleMetadataLoc = '/content/drive/My Drive/articleMetadata.csv'
resultCsvLoc = '/content/drive/My Drive/featuresOneSentence.csv'

articleData = pd.read_csv(articleMetadataLoc)



try:
  df = pd.read_csv(resultCsvLoc)
  currentLength = df.shape[0]
except:
  currentLength = 0

print(currentLength)



import pandas as pd
  

toSkip = 0

for index, row in articleData.iterrows():
  if index < currentLength:
    continue
  filename = row[1]
  print(filename)
  f = open(articleLoc + '/' + filename, "r",  encoding='cp1252')
  text = f.read()
  partitioned_string = text.partition('.')
  before_first_period = partitioned_string[0]
  print(before_first_period)
  tokens = tokenizer.encode_plus(before_first_period, max_length=MAX_LEN, padding='max_length',
                            truncation=True, return_offsets_mapping=True)
  test_tokens = np.zeros((1,MAX_LEN), dtype='int32')
  test_attention = np.zeros((1,MAX_LEN), dtype='int32')
  test_tokens[0,] = tokens['input_ids']
  test_attention[0,] = tokens['attention_mask']
  prediction = model.predict([test_tokens, test_attention])
  featureArrayStr = ''.join(map(str,cls_pooling(prediction)))
  featureArrayStr = featureArrayStr.replace('\n', '')
  with open(resultCsvLoc,'a') as fd:
    strToWrite = filename + ',' + featureArrayStr + '\n'
    fd.write(strToWrite)
  count = count + 1


0
2.txt
New York lawmakers, on average, introduce and enact more pieces of legislation than other states
3.txt
A recent Facebook post talks about a supposed scandal involving the abuse of hundreds of children in the 1980s as fact
4.txt
About 450 deaths per year are related to acetaminophen-associated overdoses, according to the FDA
5.txt
Former Secretary of State Hillary Clinton eulogized her friend and predecessor, Madeleine Albright, at Albright’s funeral service on April 27
6.txt
Twitter hasn’t said when that vote will take place, but there’s some speculation it could happen at the company’s annual meeting on May 25
7.txt
UPDATE, May 3, 2022: On April 30, two days after Mino Raiola refuted premature claims of his death, his family announced that he had died
8.txt
An image making its rounds on social media shows a purported tweet from Tesla CEO Elon Musk lamenting the logic of letting an 18-year-old amass six figures of student debt but not allowing them to acquire a business loan
9.

In [ ]:
print(len(feature_array))